In [ ]:
import cv2
import numpy as np
import copy
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage.segmentation import random_walker
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from tqdm import tqdm
from skimage.filters import threshold_multiotsu

# De-hazing Algorithm

In [ ]:


def __circularConvFilt(Img, Filter):
    FilterHeight, FilterWidth = Filter.shape
    assert (FilterHeight == FilterWidth), 'Filter must be square in shape --> Height must be same as width'
    assert (FilterHeight % 2 == 1), 'Filter dimension must be a odd number.'
    filterHalsSize = int((FilterHeight - 1) / 2)
    rows, cols = Img.shape
    PaddedImg = cv2.copyMakeBorder(Img, filterHalsSize, filterHalsSize, filterHalsSize, filterHalsSize,
                                   borderType=cv2.BORDER_WRAP)
    FilteredImg = cv2.filter2D(PaddedImg, -1, Filter)
    Result = FilteredImg[filterHalsSize:rows + filterHalsSize, filterHalsSize:cols + filterHalsSize]
    return (Result)
def __CalculateWeightingFunction(HazeImg, Filter ,sigma):
    HazeImageDouble = HazeImg.astype(float) / 255.0
    Red = HazeImageDouble[:, :, 2]
    d_r = __circularConvFilt(Red, Filter)
    Green = HazeImageDouble[:, :, 1]
    d_g = __circularConvFilt(Green, Filter)
    Blue = HazeImageDouble[:, :, 0]
    d_b = __circularConvFilt(Blue, Filter)
    return (np.exp(-((d_r ** 2) + (d_g ** 2) + (d_b ** 2)) / (2 * sigma * sigma)))
def C_CalTransmission(HazeImg , _Transmission , regularize_lambda ,sigma):
    rows, cols = _Transmission.shape
    KirschFilters = []
    KirschFilters.append(np.array([[-3, -3, -3], [-3, 0, 5], [-3, 5, 5]]))
    KirschFilters.append(np.array([[-3, -3, -3], [-3, 0, -3], [5, 5, 5]]))
    KirschFilters.append(np.array([[-3, -3, -3], [5, 0, -3], [5, 5, -3]]))
    KirschFilters.append(np.array([[5, -3, -3], [5, 0, -3], [5, -3, -3]]))
    KirschFilters.append(np.array([[5, 5, -3], [5, 0, -3], [-3, -3, -3]]))
    KirschFilters.append(np.array([[5, 5, 5], [-3, 0, -3], [-3, -3, -3]]))
    KirschFilters.append(np.array([[-3, 5, 5], [-3, 0, 5], [-3, -3, -3]]))
    KirschFilters.append(np.array([[-3, -3, 5], [-3, 0, 5], [-3, -3, 5]]))
    KirschFilters.append(np.array([[-1, -1, -1], [-1, 8, -1], [-1, -1, -1]]))

    for idx, currentFilter in enumerate(KirschFilters):
        KirschFilters[idx] = KirschFilters[idx] / np.linalg.norm(currentFilter)
    WFun = []
    for idx, currentFilter in enumerate(KirschFilters):
        WFun.append(__CalculateWeightingFunction(HazeImg, currentFilter  ,sigma))
    tF = np.fft.fft2(_Transmission)
    DS = 0
    for i in range(len(KirschFilters)):
        D = __psf2otf(KirschFilters[i], (rows, cols))
        DS = DS + (abs(D) ** 2)
    beta = 1
    beta_max = 2 ** 4
    beta_rate = 2 * np.sqrt(2)

    while (beta < beta_max):
        gamma = regularize_lambda / beta
        DU = 0
        for i in range(len(KirschFilters)):
            dt = __circularConvFilt(_Transmission, KirschFilters[i])
            u = np.maximum((abs(dt) - (WFun[i] / (len(KirschFilters) * beta))), 0) * np.sign(dt)
            DU = DU + np.fft.fft2(__circularConvFilt(u, cv2.flip(KirschFilters[i], -1)))
        _Transmission = np.abs(np.fft.ifft2((gamma * tF + DU) / (gamma + DS)))
        beta = beta * beta_rate
    return _Transmission
def __psf2otf(psf, shape):
        if np.all(psf == 0):
            return np.zeros_like(psf)
        inshape = psf.shape
        image = psf
        position='corner'
        shape = np.asarray(shape, dtype=int)
        imshape = np.asarray(image.shape, dtype=int)
        if np.alltrue(imshape == shape):
            psf= image
        else :
            dshape = shape - imshape
            pad_img = np.zeros(shape, dtype=image.dtype)
            idx, idy = np.indices(imshape)
            if position == 'center':
                offx, offy = dshape // 2
            else:
                offx, offy = (0, 0)
            pad_img[idx + offx, idy + offy] = image
            psf= pad_img
        for axis, axis_size in enumerate(inshape):
            psf = np.roll(psf, -int(axis_size / 2), axis=axis)
        otf = np.fft.fft2(psf)
        n_ops = np.sum(psf.size * np.log2(psf.shape))
        otf = np.real_if_close(otf, tol=n_ops)
        return otf
def remove_haze(HazeImg , kernel_size ,C1, C0 , regularize_lambda ,delta ,sigma,epsilon):
    AirlightEstimation_list=[]
    for ch in range(len(HazeImg.shape)):
        kernel = np.ones((kernel_size,kernel_size), np.uint8)
        eroded = cv2.erode(HazeImg[:, :, ch], kernel)
        AirlightEstimation_list.append(int(eroded.max()))
    t_b = np.maximum((AirlightEstimation_list[0] - HazeImg[:, :, 0].astype(float)) / (AirlightEstimation_list[0] - C0),
                        (HazeImg[:, :, 0].astype(float) - AirlightEstimation_list[0]) / (C1 - AirlightEstimation_list[0]))
    t_g = np.maximum((AirlightEstimation_list[1] - HazeImg[:, :, 1].astype(float)) / (AirlightEstimation_list[1] - C0),
                        (HazeImg[:, :, 1].astype(float) - AirlightEstimation_list[1]) / (C1 - AirlightEstimation_list[1]))
    t_r = np.maximum((AirlightEstimation_list[2] - HazeImg[:, :, 2].astype(float)) / (AirlightEstimation_list[2] - C0),
                        (HazeImg[:, :, 2].astype(float) - AirlightEstimation_list[2]) / (C1 - AirlightEstimation_list[2]))
    MaxVal = np.maximum(t_b, t_g, t_r)
    _Transmission = np.minimum(MaxVal, 1)
    _Transmission = C_CalTransmission(HazeImg , _Transmission , regularize_lambda ,sigma)
    Transmission = pow(np.maximum(abs(_Transmission), epsilon), delta)
    HazeCorrectedImage = copy.deepcopy(HazeImg)
    for ch in range(len(HazeImg.shape)):
        temp = ((HazeImg[:, :, ch].astype(float) - AirlightEstimation_list[ch]) / Transmission) + AirlightEstimation_list[ch]
        temp = np.maximum(np.minimum(temp, 255), 0)
        HazeCorrectedImage[:, :, ch] = temp
    haze_corrected_img = HazeCorrectedImage
    HazeTransmissionMap = _Transmission
    return (haze_corrected_img, HazeTransmissionMap)




# Preprocess & Postprocess

In [ ]:
def find_mask(IMG_PATH):
    image = cv2.imread(IMG_PATH)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_sky = np.array([50, 0, 200])  # Adjust these values based your specific sky color
    upper_sky = np.array([170, 255, 255])
    mask = cv2.inRange(hsv_image, lower_sky, upper_sky)
    kernel =np.ones((3, 3),np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)

    mask[int(2*mask.shape[0]/3):,:]=0
    mask = cv2.erode(mask, kernel, iterations=30)
    mask = cv2.dilate(mask, kernel, iterations=5)
    mask[mask==1] = 255
    dst = cv2.GaussianBlur(mask,(9,9),cv2.BORDER_DEFAULT)
    for i in range(200):
        dst = cv2.GaussianBlur(dst,(5,5),cv2.BORDER_DEFAULT)
    final_mask = cv2.merge([dst,dst,dst])
    return final_mask


def find_dehazing_mask(IMG_PATH):
    image = cv2.imread(IMG_PATH)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    lower_sky = np.array([50, 0, 200])  # Adjust these values based your specific sky color
    upper_sky = np.array([170, 255, 255])

    mask = cv2.inRange(hsv_image, lower_sky, upper_sky)

    kernel =np.ones((3, 3),np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)

    mask[int(2*mask.shape[0]/3):,:]=0

    ground_mask = cv2.erode(mask, kernel, iterations=60)
    ground_mask = cv2.bitwise_not(ground_mask)
    _, thresholded = cv2.threshold(ground_mask, 127, 1, cv2.THRESH_BINARY)
    ground=cv2.bitwise_and(image, image, mask=thresholded)
    #show(ground)
    sky_mask = cv2.dilate(mask, kernel, iterations=50)
    _, thresholded = cv2.threshold(sky_mask, 127, 1, cv2.THRESH_BINARY)
    sky=cv2.bitwise_and(image, image, mask=thresholded)
    #show(sky)
    return (ground, sky)


# Final Results

In [ ]:

# for i in np.arange(5,450,5):
IMG_PATH = 'IMAGE PATH'
ground, sky = find_dehazing_mask(IMG_PATH)
HazeCorrectedImg, haze_map = remove_haze(ground , kernel_size=15 ,C1=300 , C0=20 , regularize_lambda=0.1 ,delta=0.75 , sigma=0.5,epsilon=0.0001)
HazeCorrectedImg = HazeCorrectedImg.astype(np.int32)
image = cv2.imread(IMG_PATH)
mask = find_mask(IMG_PATH)
new_mask = mask.astype(np.int32)
image = image.astype(np.int32)
sky_img = (np.multiply(image, new_mask) / 255).astype(np.uint8)
mask_inv = ((np.ones_like(new_mask) * 255) - new_mask).astype(np.int32)
masked_img = (np.multiply(HazeCorrectedImg, mask_inv)/255).astype(np.uint8)
final_image = sky_img + masked_img



# cv2.imwrite('path', final_image)
